# First create a simulated dataset representing an experimental target dataset.

In [2]:
from ase.lattice.cubic import FaceCenteredCubic, SimpleCubic, BodyCenteredCubic
from ase.lattice.hexagonal import HexagonalClosedPacked
from ase.cluster.icosahedron import Icosahedron
from ase.cluster.decahedron import Decahedron
from ase.cluster.octahedron import Octahedron
from debyecalculator import DebyeCalculator
import torch
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots

def calculate_pdf(cluster, function='Gr'):
    """
    Calculate a PDF or Sq from a structure.

    Parameters:
    cluster: The structure to calculate the PDF or Sq from.
    function (str): The function to calculate. 'Gr' for pair distribution function, 'Sq' for structure factor. Default is 'Gr'.

    Returns:
    r (list): The r values from the calculated PDF or Sq.
    G (list): The G values from the calculated PDF or Sq.
    """
    # Check if the function parameter is valid
    assert function in ['Gr', 'Sq'], "Function must be 'Gr' or 'Sq'"

    # Extract atomic symbols and positions
    symbols = cluster.get_chemical_symbols()
    positions = cluster.get_positions()

    # Convert positions to a torch tensor
    positions_tensor = torch.tensor(positions)

    # Create a structure tuple
    structure_tuple = (symbols, positions_tensor)

    # Calculate Pair Distribution Function or Structure Factor
    if function == 'Gr':
        r, G = calc.gr(structure_source=structure_tuple)
    else:  # function == 'Sq'
        r, G = calc.sq(structure_source=structure_tuple)

    G /= G.max()

    return r, G

# Create the subplots
fig = make_subplots(rows=2, cols=1)

# Initialise calculator object
calc = DebyeCalculator(qmin=2, qmax=10.0, rmax=30, qstep=0.01)

# Create a cluster
latticeconstant = 3.99  # Lattice constant
noshells = 7  # Number of shells
element = 'Au'  # Element symbol
cluster = Icosahedron(element, noshells, latticeconstant)

# Calculate the structure factor and plot it
q, S = calculate_pdf(cluster, function="Sq")
fig.add_trace(go.Scatter(x=q, y=S, mode='lines', name='Icosahedron, noshells=7'), row=1, col=1)

# Calculate the pair distribution function and plot it
r, G = calculate_pdf(cluster, function="Gr")
fig.add_trace(go.Scatter(x=r, y=G, mode='lines', name='Icosahedron, noshells=7'), row=2, col=1)

# Update layout
fig.update_layout(title='Calculated PDF and Sq', xaxis_title='r (Å)', yaxis_title='G (a. u.)', xaxis2_title='q (Å⁻¹)', yaxis2_title='S(q)')

# Show the plot
fig.show()

# Save r and G
np.save('../Data/Gr/Target_PDF_benchmark.npy', np.vstack((r, G)).T)

# Save r and S
np.save('../Data/Sq/Target_Sq_benchmark.npy', np.vstack((q, S)).T)


/var/folders/pg/s9x5c7555vb2c5gjlpnlzqyh0000gp/T/ipykernel_44880/643611181.py:52: UserWarning:


